### import and read data

In [1]:
import json
import math
import os

import numpy as np
import pandas as pd
from keras import layers
from tensorflow import keras
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf



In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if physical_devices:
    # Nếu có GPU, in ra thông tin của từng GPU
    for gpu in physical_devices:
        # Lấy thông tin chi tiết của GPU
        gpu_details = tf.config.experimental.get_device_details(gpu)
        
        # # In ra tên và dung lượng RAM của GPU
        print("GPU Device:", gpu_details["device_name"])
else:
    print("No GPU available, TensorFlow is using CPU.")


GPU Device: GeForce RTX 2070


In [3]:

# mount google drive
WORK_PATH = 'C:/Users/ADMIN/Contacts/KLTN/data'
SAVE_PATH = 'C:/Users/ADMIN/Contacts/KLTN/model'

In [4]:

# batch_size = 16
batch_size = 64
IMG_HEIGHT = int(224)
IMG_WIDTH = int(224)

In [5]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
i_train = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=os.path.join(WORK_PATH,'train_balan'),
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH))



Found 2305 images belonging to 5 classes.


In [6]:
# columns = ["model_name", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
# df = pd.DataFrame(columns=columns)
# df.to_csv("history_choice.csv", index=False)
# pd.read_csv('history_choice.csv')

### VGG19

In [7]:
def build_model_vgg19():
    inputs = keras.layers.Input((IMG_HEIGHT,IMG_WIDTH,3), name='inputs')
    model = keras.applications.VGG19(include_top= False, weights= 'imagenet', input_tensor= inputs)
    x = model.output
    x = keras.layers.Flatten(name='flatten')(x)
    x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_1')(x)
    x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_2')(x)
    outputs = keras.layers.Dense(5,activation= 'softmax', name='dense_out')(x)
    model = keras.Model(inputs=inputs, outputs = outputs,name="model_use_VGG19")
    return model

In [ ]:
model = build_model_vgg19()
# Biên dịch
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001,
                                                amsgrad=True),
              loss='categorical_crossentropy',metrics='accuracy')

# callbacks_1 = keras.callbacks.EarlyStopping(monitor="val_accuracy",patience = 10,
#                                           verbose=1,min_delta = 0.00001,
#                                           restore_best_weights=True)

callbacks_2 = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(SAVE_PATH,'Model_VGG19_v1.h5'),
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)
callbacks = [callbacks_2]
#Training model
history = model.fit(i_train,epochs = 30,callbacks = callbacks)


In [ ]:
columns = ["model_name", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
history_df = pd.DataFrame(history.history)
history_df['model_name'] = model.name
history_df['epoch'] = history_df.index+1
history_df= history_df.reindex(columns=columns)
history_df.to_csv('history_choice.csv', index=False,mode='a',header = False)

### VGG16

In [ ]:
def build_model_vgg16():
    inputs = keras.layers.Input((IMG_HEIGHT,IMG_WIDTH,3), name='inputs')
    model = keras.applications.VGG16(include_top= False, weights= 'imagenet', input_tensor= inputs)
    x = model.output
    x = keras.layers.Flatten(name='flatten')(x)
    x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_1')(x)
    x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_2')(x)
    outputs = keras.layers.Dense(5,activation= 'softmax', name='dense_out')(x)
    model = keras.Model(inputs=inputs, outputs = outputs,name="model_use_VGG16")
    return model

In [ ]:
model = build_model_vgg16()
# Biên dịch
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001,
                                                amsgrad=True),
              loss='categorical_crossentropy',metrics='accuracy')

# callbacks_1 = keras.callbacks.EarlyStopping(monitor="val_accuracy",patience = 10,
#                                           verbose=1,min_delta = 0.00001,
#                                           restore_best_weights=True)

callbacks_2 = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(SAVE_PATH,'Model_VGG16_v1.h5'),
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)
callbacks = [callbacks_2]
#Training model
history = model.fit(i_train,epochs = 30,callbacks = callbacks)

In [ ]:
columns = ["model_name", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
history_df = pd.DataFrame(history.history)
history_df['model_name'] = model.name
history_df['epoch'] = history_df.index+1
history_df= history_df.reindex(columns=columns)
history_df.to_csv('history_choice.csv', index=False,mode='a',header = False)

### Resnet50

In [9]:
def build_model_resnet50():
    inputs = keras.layers.Input((IMG_HEIGHT,IMG_WIDTH,3), name='inputs')
    model = keras.applications.resnet50.ResNet50(include_top= False, weights= 'imagenet', input_tensor= inputs)
    x = model.output
    x = keras.layers.GlobalAvgPool2D()(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(5,activation= 'softmax')(x)
    model = keras.Model(inputs=inputs, outputs = outputs,name="model_use_ResNet50")
    # Đóng băng 1 phần mô hình
    l = len(model.layers)
    for i, layer in enumerate(model.layers):
        if i <= 0.7*l: layer.trainable = False
    return model



In [ ]:
model = build_model_resnet50()
# Biên dịch
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001,
                                                amsgrad=True),
              loss='categorical_crossentropy',metrics='accuracy')

# callbacks_1 = keras.callbacks.EarlyStopping(monitor="val_accuracy",patience = 10,
#                                           verbose=1,min_delta = 0.00001,
#                                           restore_best_weights=True)

callbacks_2 = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(SAVE_PATH,'Model_resnet50_v1.h5'),
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)
callbacks = [callbacks_2]
#Training model



In [ ]:
history = model.fit(i_train,epochs = 20,callbacks = callbacks)


In [ ]:
columns = ["model_name", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
history_df = pd.DataFrame(history.history)
history_df['model_name'] = model.name
history_df['epoch'] = history_df.index+1
history_df= history_df.reindex(columns=columns)
history_df.to_csv('history_choice.csv', index=False,mode='a',header = False)

In [ ]:
history = model.fit(i_train,epochs = 50,callbacks = callbacks)


In [ ]:
columns = ["model_name", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
history_df = pd.DataFrame(history.history)
history_df['model_name'] = model.name
history_df['epoch'] = history_df.index+21
history_df= history_df.reindex(columns=columns)
history_df.to_csv('history_choice.csv', index=False,mode='a',header = False)

### Resnet101

In [11]:
def build_model_resnet101():
    inputs = keras.layers.Input((IMG_HEIGHT,IMG_WIDTH,3), name='inputs')
    model = tf.keras.applications.ResNet101(include_top= False, weights= 'imagenet', input_tensor= inputs)
    x = model.output
    x = keras.layers.GlobalAvgPool2D()(x)
    x = keras.layers.Dropout(0.7)(x)
    outputs = keras.layers.Dense(5,activation= 'softmax')(x)
    model = keras.Model(inputs=inputs, outputs = outputs,name="model_use_ResNet101")
    # Đóng băng 1 phần mô hình
    l = len(model.layers)
    for i, layer in enumerate(model.layers):
        if i <= 0.8*l: layer.trainable = False
    return model


In [ ]:
model = build_model_resnet101()
# Biên dịch
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001,
                                                amsgrad=True),
              loss='categorical_crossentropy',metrics='accuracy')

# callbacks_1 = keras.callbacks.EarlyStopping(monitor="val_accuracy",patience = 10,
#                                           verbose=1,min_delta = 0.00001,
#                                           restore_best_weights=True)

callbacks_2 = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(SAVE_PATH,'Model_Resnet101_v1.h5'),
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)
callbacks = [callbacks_2]
#Training model



In [9]:
history = model.fit(i_train,epochs = 70,callbacks = callbacks)


Epoch 1/70
73/73 [==============================] - 23s 198ms/step - loss: 1.7479 - accuracy: 0.2095
Epoch 2/70
73/73 [==============================] - 18s 250ms/step - loss: 1.6023 - accuracy: 0.2651
Epoch 3/70
73/73 [==============================] - 19s 255ms/step - loss: 1.5558 - accuracy: 0.3098
Epoch 4/70
73/73 [==============================] - 13s 183ms/step - loss: 1.5463 - accuracy: 0.3046
Epoch 5/70
73/73 [==============================] - 19s 259ms/step - loss: 1.5008 - accuracy: 0.3197
Epoch 6/70
73/73 [==============================] - 13s 183ms/step - loss: 1.5106 - accuracy: 0.3128
Epoch 7/70
73/73 [==============================] - 19s 255ms/step - loss: 1.4891 - accuracy: 0.3310
Epoch 8/70
73/73 [==============================] - 19s 258ms/step - loss: 1.4687 - accuracy: 0.3341
Epoch 9/70
73/73 [==============================] - 18s 253ms/step - loss: 1.4449 - accuracy: 0.3631
Epoch 10/70
73/73 [==============================] - 13s 184ms/step - loss: 1.4349 - accura

In [10]:
columns = ["model_name", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
history_df = pd.DataFrame(history.history)
history_df['model_name'] = model.name
history_df['epoch'] = history_df.index+1
history_df= history_df.reindex(columns=columns)
history_df.to_csv('history_choice.csv', index=False,mode='a',header = False)

In [11]:
pd.read_csv('history_choice.csv')

,model_name,epoch,loss,accuracy,val_loss,val_accuracy
0,model_use_VGG19,1,1.677625,0.213449,NaN,NaN
1,model_use_VGG19,2,1.608900,0.206941,NaN,NaN
2,model_use_VGG19,3,1.608447,0.213449,NaN,NaN
3,model_use_VGG19,4,1.608480,0.211280,NaN,NaN
4,model_use_VGG19,5,1.608510,0.210412,NaN,NaN
...,...,...,...,...,...,...
195,model_use_ResNet101,66,0.179961,0.934924,NaN,NaN
196,model_use_ResNet101,67,0.146534,0.951844,NaN,NaN
197,model_use_ResNet101,68,0.132230,0.959653,NaN,NaN
198,model_use_ResNet101,69,0.145685,0.948807,NaN,NaN


### Vgg19_v2

In [6]:
def vgg19_return():
    inputs = keras.layers.Input((IMG_HEIGHT,IMG_WIDTH,3), name='inputs')
    model = keras.applications.VGG19(include_top= False, weights= 'imagenet', input_tensor= inputs)
    x = model.output
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation='relu', name='dense_6')(x)
    x = keras.layers.Dense(128, activation='relu', name='dense_7')(x)
    x = keras.layers.Dense(64, activation='relu', name='dense_8')(x)
    x = keras.layers.Dense(32, activation='relu', name='dense_9')(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(5,activation= 'softmax', name='dense_out')(x)
    model = keras.Model(inputs=inputs, outputs = outputs,name="model_use_VGG19_addDense256-128-64-32")
    l = len(model.layers)
    for i, layer in enumerate(model.layers):
        if i <= 0.5*l: layer.trainable = False
    model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001,
                                                amsgrad=True),
              loss='categorical_crossentropy',metrics='accuracy')
    return model

In [7]:
"Flatten, Dense 256-128-64-32, Dropout(0.5)"

'Flatten, Dense 256-128-64-32, Dropout(0.5)'

In [126]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rescale=1./255) # Generator for our training data
i_train = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=os.path.join(WORK_PATH,'train_balan'),
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH))



Found 2305 images belonging to 5 classes.


In [129]:
model = vgg19_return()
callbacks_1 = keras.callbacks.EarlyStopping(monitor="accuracy",patience = 10,
                                          verbose=1,min_delta = 0.00001,
                                          restore_best_weights=True)

callbacks_2 = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(SAVE_PATH,'Model_VGG19_v1_2.h5'),
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)
callbacks = [callbacks_2]



In [130]:
history = model.fit(i_train,epochs = 100,callbacks = callbacks)


Epoch 1/100
37/37 [==============================] - 15s 384ms/step - loss: 1.6240 - accuracy: 0.2143
Epoch 2/100
37/37 [==============================] - 13s 357ms/step - loss: 1.5856 - accuracy: 0.2568
Epoch 3/100
37/37 [==============================] - 13s 358ms/step - loss: 1.5272 - accuracy: 0.2885
Epoch 4/100
37/37 [==============================] - 14s 377ms/step - loss: 1.5052 - accuracy: 0.3072
Epoch 5/100
37/37 [==============================] - 13s 357ms/step - loss: 1.4372 - accuracy: 0.3492
Epoch 6/100
37/37 [==============================] - 13s 350ms/step - loss: 1.4317 - accuracy: 0.3367
Epoch 7/100
37/37 [==============================] - 13s 339ms/step - loss: 1.4399 - accuracy: 0.3423
Epoch 8/100
37/37 [==============================] - 13s 360ms/step - loss: 1.3733 - accuracy: 0.3562
Epoch 9/100
37/37 [==============================] - 13s 362ms/step - loss: 1.3619 - accuracy: 0.3731
Epoch 10/100
37/37 [==============================] - 13s 363ms/step - loss: 1.335

In [131]:
columns = ["model_name", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
history_df = pd.DataFrame(history.history)
history_df['model_name'] = model.name
history_df['epoch'] = history_df.index+1
history_df= history_df.reindex(columns=columns)
history_df.to_csv('history_choice.csv', index=False,mode='a',header = False)

In [ ]:
# # luư lịch sử chạy các model
# columns = ["model_name","layer","fold", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
# df = pd.DataFrame(columns=columns)
# df.to_csv("history_fit.csv", index=False)
# pd.read_csv('history_fit.csv')

In [ ]:
test_image_generator = ImageDataGenerator(rescale=1./255)

i_test = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=os.path.join(WORK_PATH,'test'),
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH))
result = model.evaluate(i_test)